This notebook performs profile feature selection.

In [1]:
import argparse
import pathlib
import sys

import pandas as pd
from pycytominer import feature_select

cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd
else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break
sys.path.append(str(root_dir / "utils"))
from arg_parsing_utils import parse_args
from notebook_init_utils import bandicoot_check, init_notebook

root_dir, in_notebook = init_notebook()

profile_base_dir = bandicoot_check(pathlib.Path("~/mnt/bandicoot").resolve(), root_dir)

In [2]:
if not in_notebook:
    args = parse_args()
    patient = args["patient"]

else:
    patient = "NF0014_T1"

In [3]:
# pathing
sc_normalized_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/image_based_profiles/3.normalized_profiles/sc_norm.parquet"
).resolve(strict=True)
organoid_normalized_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/image_based_profiles/3.normalized_profiles/organoid_norm.parquet"
).resolve(strict=True)


# output path
sc_fs_output_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/image_based_profiles/4.feature_selected_profiles/sc_fs.parquet"
).resolve()
organoid_fs_output_path = pathlib.Path(
    f"{profile_base_dir}/data/{patient}/image_based_profiles/4.feature_selected_profiles/organoid_fs.parquet"
).resolve()

organoid_fs_output_path.parent.mkdir(parents=True, exist_ok=True)

In [4]:
# read in the data
sc_normalized = pd.read_parquet(sc_normalized_path)
organoid_normalized = pd.read_parquet(organoid_normalized_path)

In [5]:
feature_select_ops = [
    "drop_na_columns",
    "blocklist",
    # "correlation_threshold", # comment out to remove correlation thresholding
    # "variance_threshold", # comment out to remove variance thresholding
]

In [6]:
na_cutoff = 0.05
corr_threshold = 0.95
freq_cut = 0.01
unique_cut = 0.01

### Feature select the single-cell profiles

In [7]:
sc_normalized.head()

,Metadata_patient_tumor,Metadata_object_id,Metadata_unit,Metadata_dose,Metadata_treatment,Metadata_image_set,Metadata_Well,Metadata_parent_organoid,Metadata_Target,Metadata_Class,...,Intensity_Cytoplasm_Mito_MAX.Z_y,Intensity_Cytoplasm_Mito_MEAN.INTENSITY_y,Intensity_Cytoplasm_Mito_MEAN.INTENSITY.EDGE_y,Intensity_Cytoplasm_Mito_MEDIAN.INTENSITY_y,Intensity_Cytoplasm_Mito_MIN.INTENSITY_y,Intensity_Cytoplasm_Mito_MIN.INTENSITY.EDGE_y,Intensity_Cytoplasm_Mito_STD.INTENSITY_y,Intensity_Cytoplasm_Mito_STD.INTENSITY.EDGE_y,Intensity_Cytoplasm_Mito_UPPER.QUARTILE.INTENSITY_y,Intensity_Cytoplasm_Mito_VOLUME_y
0,NF0014_T1,255,uM,1,Fimepinostat,E5-2,E5,19,PI3K and HDAC inhibitor,Small Molecule,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NF0014_T1,25,uM,1,Fimepinostat,D5-2,D5,-1,PI3K and HDAC inhibitor,Small Molecule,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NF0014_T1,51,uM,1,Fimepinostat,D5-2,D5,-1,PI3K and HDAC inhibitor,Small Molecule,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NF0014_T1,63,uM,1,Fimepinostat,D5-2,D5,-1,PI3K and HDAC inhibitor,Small Molecule,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NF0014_T1,76,uM,1,Fimepinostat,D5-2,D5,-1,PI3K and HDAC inhibitor,Small Molecule,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
sc_blocklist = [
    x
    for x in sc_normalized.columns
    if "Area" in x and ("MAX" in x or "MIN" in x or "BBOX" in x or "CENTER" in x)
]
sc_blocklist += [
    x
    for x in sc_normalized.columns
    if "Intensity" in x
    and (
        "MIN.X" in x
        or "MAX.X" in x
        or "MIN.Y" in x
        or "MAX.Y" in x
        or "MIN.Z" in x
        or "MAX.Z" in x
    )
]
# write the blocklist to a file
# add "blocklist" the beginning of the list
sc_blocklist = ["blocklist"] + sc_blocklist
sc_blocklist_path = pathlib.Path(
    f"{root_dir}/4.processing_image_based_profiles/data/blocklist/sc_blocklist.txt"
).resolve()
sc_blocklist_path.parent.mkdir(parents=True, exist_ok=True)
with open(sc_blocklist_path, "w") as f:
    for item in sc_blocklist:
        f.write(f"{item}\n")

In [9]:
sc_metadata_columns = [x for x in sc_normalized.columns if "Metadata" in x]

sc_metadata_columns += [
    "Area.Size.Shape_Cell_CENTER.X",
    "Area.Size.Shape_Cell_CENTER.Y",
    "Area.Size.Shape_Cell_CENTER.Z",
]
sc_features_columns = [
    col for col in sc_normalized.columns if col not in sc_metadata_columns
]
all_trt_df = sc_normalized.copy()

sc_normalized = sc_normalized.loc[
    sc_normalized["Metadata_treatment"].isin(["DMSO", "Staurosporine"])
]

sc_features_columns = [
    col for col in sc_normalized.columns if col not in sc_metadata_columns
]
sc_features_df = sc_normalized.drop(columns=sc_metadata_columns, errors="ignore")

In [10]:
# fs the data
sc_fs_profiles = feature_select(
    sc_features_df,
    operation=feature_select_ops,
    features=sc_features_columns,
    blocklist_file=sc_blocklist_path,
    na_cutoff=na_cutoff,
    corr_threshold=corr_threshold,
    freq_cut=freq_cut,
    unique_cut=unique_cut,
)
fs_profiles = all_trt_df[
    [col for col in all_trt_df.columns if col in sc_fs_profiles.columns]
]

original_data_shape = sc_normalized.shape
sc_fs_profiles = pd.concat(
    [
        all_trt_df[sc_metadata_columns].reset_index(drop=True),
        sc_fs_profiles.reset_index(drop=True),
    ],
    axis=1,
)
print("The number features before feature selection:", original_data_shape[1])
print("The number features after feature selection:", sc_fs_profiles.shape[1])
sc_fs_profiles.to_parquet(sc_fs_output_path, index=False)
sc_fs_profiles.head()

The number features before feature selection: 1880
The number features after feature selection: 1281


,Metadata_patient_tumor,Metadata_object_id,Metadata_unit,Metadata_dose,Metadata_treatment,Metadata_image_set,Metadata_Well,Metadata_parent_organoid,Metadata_Target,Metadata_Class,...,Texture_Cytoplasm_Mito_Difference.Entropy_256.3,Texture_Cytoplasm_Mito_Difference.Variance_256.3,Texture_Cytoplasm_Mito_Entropy_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Cytoplasm_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Cytoplasm_Mito_Inverse.Difference.Moment_256.3,Texture_Cytoplasm_Mito_Sum.Average_256.3,Texture_Cytoplasm_Mito_Sum.Entropy_256.3,Texture_Cytoplasm_Mito_Sum.Variance_256.3,Texture_Cytoplasm_Mito_Variance_256.3
0,NF0014_T1,255,uM,1,Fimepinostat,E5-2,E5,19,PI3K and HDAC inhibitor,Small Molecule,...,0.239311,-0.127454,0.168712,-1.395844,1.089556,-0.113750,0.598492,0.194094,1.482163,1.461434
1,NF0014_T1,25,uM,1,Fimepinostat,D5-2,D5,-1,PI3K and HDAC inhibitor,Small Molecule,...,-0.007093,0.034392,-0.025312,-0.979456,0.387830,0.035037,0.224652,-0.018084,0.921696,0.942498
2,NF0014_T1,51,uM,1,Fimepinostat,D5-2,D5,-1,PI3K and HDAC inhibitor,Small Molecule,...,-0.151367,0.126512,-0.136218,-0.348196,-0.148967,0.120089,-0.087394,-0.141902,0.149061,0.171092
3,NF0014_T1,63,uM,1,Fimepinostat,D5-2,D5,-1,PI3K and HDAC inhibitor,Small Molecule,...,0.182480,-0.098700,0.130216,-1.293351,0.954294,-0.087870,0.724839,0.139665,2.285482,2.302174
4,NF0014_T1,76,uM,1,Fimepinostat,D5-2,D5,-1,PI3K and HDAC inhibitor,Small Molecule,...,-0.117540,0.105099,-0.113870,-0.336922,-0.066265,0.100326,-0.030359,-0.116770,0.264447,0.300243


### Normalize the organoid profiles

In [11]:
organoid_normalized.head()

,Metadata_patient_tumor,Metadata_object_id,Metadata_unit,Metadata_dose,Metadata_treatment,Metadata_image_set,Metadata_Well,Metadata_single_cell_count,Metadata_Target,Metadata_Class,...,Texture_Organoid_Mito_Difference.Entropy_256.3,Texture_Organoid_Mito_Difference.Variance_256.3,Texture_Organoid_Mito_Entropy_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.1_256.3,Texture_Organoid_Mito_Information.Measure.of.Correlation.2_256.3,Texture_Organoid_Mito_Inverse.Difference.Moment_256.3,Texture_Organoid_Mito_Sum.Average_256.3,Texture_Organoid_Mito_Sum.Entropy_256.3,Texture_Organoid_Mito_Sum.Variance_256.3,Texture_Organoid_Mito_Variance_256.3
0,NF0014_T1,19,uM,1,Fimepinostat,E5-2,E5,1.0,PI3K and HDAC inhibitor,Small Molecule,...,-0.744079,0.929269,-0.876963,-0.149888,-0.453953,0.924721,-1.012954,-0.840821,-0.826647,-0.812516
1,NF0014_T1,15,uM,1,Fimepinostat,D5-2,D5,11.0,PI3K and HDAC inhibitor,Small Molecule,...,-0.018758,0.263903,-0.497285,1.297494,-1.614111,0.248123,-1.166291,-0.386110,-1.094204,-1.082140
2,NF0014_T1,47,uM,1,Ketotifen,G6-1,G6,9.0,histamine H1 receptor antagonist,Small Molecule,...,-0.825789,0.752942,-0.748346,0.050014,-0.465444,0.661318,-0.523751,-0.848608,-0.527859,-0.511635
3,NF0014_T1,40,uM,10,Mirdametinib,G8-1,G8,8.0,MEK1/2 inhibitor,Small Molecule,...,-1.928917,1.998597,-1.865807,-0.723970,-1.483790,2.002970,-1.380694,-1.952115,-0.974235,-0.986668
4,NF0014_T1,37,uM,10,Mirdametinib,C9-2,C9,13.0,MEK1/2 inhibitor,Small Molecule,...,1.306989,-0.622370,1.532605,0.813374,0.531467,-0.458024,-0.573092,1.218386,-0.648673,-0.629151


In [12]:
organoid_blocklist = [
    x
    for x in organoid_normalized.columns
    if "Area" in x and ("MAX" in x or "MIN" in x or "BBOX" in x or "CENTER" in x)
]
organoid_blocklist += [
    x
    for x in organoid_normalized.columns
    if "Intensity" in x
    and (
        "MIN.X" in x
        or "MAX.X" in x
        or "MIN.Y" in x
        or "MAX.Y" in x
        or "MIN.Z" in x
        or "MAX.Z" in x
    )
]
# write the blocklist to a file
# add "blocklist" the beginning of the list
organoid_blocklist = ["blocklist"] + organoid_blocklist
organoid_blocklist_path = pathlib.Path(
    "../data/blocklist/organoid_blocklist.txt"
).resolve()
organoid_blocklist_path.parent.mkdir(parents=True, exist_ok=True)
with open(organoid_blocklist_path, "w") as f:
    for item in organoid_blocklist:
        f.write(f"{item}\n")

In [13]:
organoid_metadata_columns = [x for x in organoid_normalized.columns if "Metadata" in x]
organoid_metadata_columns += [
    "Area.Size.Shape_Organoid_CENTER.X",
    "Area.Size.Shape_Organoid_CENTER.Y",
    "Area.Size.Shape_Organoid_CENTER.Z",
]
organoid_features_columns = [
    col for col in organoid_normalized.columns if col not in organoid_metadata_columns
]
all_trt_df = organoid_normalized.copy()
organoid_normalized = organoid_normalized.loc[
    organoid_normalized["Metadata_treatment"].isin(["DMSO", "Staurosporine"])
]
organoid_features_columns = [
    col for col in organoid_normalized.columns if col not in organoid_metadata_columns
]
organoid_features_df = organoid_normalized.drop(
    columns=organoid_metadata_columns, errors="ignore"
)

In [14]:
# normalize the data
organoid_fs_profiles = feature_select(
    organoid_features_df,
    operation=feature_select_ops,
    features=organoid_features_columns,
    blocklist_file=organoid_blocklist_path,
    na_cutoff=na_cutoff,
    corr_threshold=corr_threshold,
    freq_cut=freq_cut,
    unique_cut=unique_cut,
)
# apply feature selection to all profiles
organoid_fs_profiles = all_trt_df[
    [col for col in all_trt_df.columns if col in organoid_fs_profiles.columns]
]
# concatenate the metadata and the feature selected profiles
original_data_shape = organoid_normalized.shape
organoid_fs_profiles = pd.concat(
    [
        all_trt_df[organoid_metadata_columns].reset_index(drop=True),
        organoid_fs_profiles.reset_index(drop=True),
    ],
    axis=1,
)

print("The number features before feature selection:", original_data_shape[1])
print("The number features after feature selection:", organoid_fs_profiles.shape[1])
organoid_fs_profiles.to_parquet(organoid_fs_output_path, index=False)
organoid_fs_profiles.head()

The number features before feature selection: 464
The number features after feature selection: 16


,Metadata_patient_tumor,Metadata_object_id,Metadata_unit,Metadata_dose,Metadata_treatment,Metadata_image_set,Metadata_Well,Metadata_single_cell_count,Metadata_Target,Metadata_Class,Metadata_Therapeutic_Categories,Metadata_patient,Metadata_tumor,Area.Size.Shape_Organoid_CENTER.X,Area.Size.Shape_Organoid_CENTER.Y,Area.Size.Shape_Organoid_CENTER.Z
0,NF0014_T1,19,uM,1,Fimepinostat,E5-2,E5,1.0,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1,938.570068,944.124634,4.906299
1,NF0014_T1,15,uM,1,Fimepinostat,D5-2,D5,11.0,PI3K and HDAC inhibitor,Small Molecule,Investigational,NF0014,T1,950.582825,586.024475,10.748533
2,NF0014_T1,47,uM,1,Ketotifen,G6-1,G6,9.0,histamine H1 receptor antagonist,Small Molecule,Anti-Allergic Agents,NF0014,T1,795.807983,995.676208,8.956269
3,NF0014_T1,40,uM,10,Mirdametinib,G8-1,G8,8.0,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor,NF0014,T1,769.689636,720.108276,16.487421
4,NF0014_T1,37,uM,10,Mirdametinib,C9-2,C9,13.0,MEK1/2 inhibitor,Small Molecule,Kinase Inhibitor,NF0014,T1,817.411133,834.836914,9.496585
